# 1. Raw Data

### 1.1 Create DB Conncection

In [ ]:
import db_connect
sql = db_connect.DatabaseConnect('CS581GROUP6','localhost','root','qwerty','3306')

# 2. Preprocessing

In [ ]:
from datetime import datetime
from datetime import timedelta  
import numpy as np
import pandas as pd
import math
import sys
from geopy.distance import geodesic
import requests
import json
import networkx as nx

In [ ]:
max_delay_percent = 0.20
max_delay = 600
max_walking_time_percent = 0.10
max_walking_time = 240
#AG - UIC
#graphhoper_key_test = '5d92bf2d-b97c-47a4-a93f-0169d1d09a40'
#GP - UIC
#graphhoper_key_test = '69f69e94-6f53-4436-89c0-1d98b89c8707'
#GP - ME
#graphhoper_key_test = 'd407f98d-bfbe-475c-bf64-af45085528c5'
#GP - ICLOUD
graphhoper_key_test = 'bbf822ce-0fed-4827-a4c6-c697cc97c6cc'
pool_duration = 5
api_hits = 0
destinationDict = {}

filepath = 'RideSharingData/filtered_data_Jan2016.csv'
#filepath = 'RideSharingData/filtered_data_2016-03-10.csv'
#filepath = 'RideSharingData/filtered_data_2016-03-10_No-Walk.csv'

In [ ]:
#GP - GMAIL
#graphhoper_key_test = '7b0f35c6-6c30-4fb2-8572-23be956e770c'
#AG - DATASET B
#graphhoper_key_test = '01f52585-e7bd-46ec-bdfa-e2c118d6c08b'

### 2.1 Mapping Coordinates to Destination ID

In [ ]:
def getdestid(lat,longt):
    return df_destinations.iloc[df_destinations.apply(lambda x: np.linalg.norm(np.array([lat, longt]) - np.array([x['destLat'],x['destLong']])), axis=1).idxmin(axis = 0)]['destID']

In [ ]:
def search_map_dest_id(lat_start, lat_end, long_start, long_end, dest_lat, dest_long, num_of_iter, num_of_iter_limit):
    
    if lat_start + (lat_end - lat_start)/2 >= dest_lat:
        lat_end = lat_start + (lat_end - lat_start)/2
    else:
        lat_start = lat_start + (lat_end - lat_start)/2

    if long_start + (long_end - long_start)/2 >= dest_long:
        long_end = long_start + (long_end - long_start)/2
    else:
        long_start = long_start + (long_end - long_start)/2
    num_of_iter = num_of_iter + 1
    if num_of_iter <= num_of_iter_limit:
        return search_map_dest_id(lat_start, lat_end, long_start, long_end, dest_lat, dest_long, num_of_iter,num_of_iter_limit)

    else:
        filtered_dest = df_destinations.loc[(df_destinations['destLat'] >= lat_start) & (df_destinations['destLat'] <= lat_end) & (df_destinations['destLong'] >= long_start) & (df_destinations['destLong'] <= long_end)]
        if len(filtered_dest) == 0:
            return False
        else:    
            return df_destinations.iloc[filtered_dest.apply(lambda x: np.linalg.norm(np.array([dest_lat,dest_long]) - np.array([x['destLat'],x['destLong']])), axis=1).idxmin(axis = 0)]['destID']

### 2.2 Pool Window Assignment

In [ ]:
def getPoolWindow(pickup_time, pool_start_time, pool_window_id):
    
    pool_end_time = pool_start_time + timedelta(minutes = pool_duration)
    
    if(pickup_time > pool_end_time):
        
        pool_window_id += 1
    
        while(pickup_time > pool_end_time):

            pool_start_time += timedelta(minutes = pool_duration)
            pool_end_time = pool_start_time + timedelta(minutes = pool_duration)
                
    return pool_start_time, pool_window_id

def poolWindowAssignment():
    
    pool_window_id = 1 
    pool_start_time = df['tpep_pickup_datetime'][0]
    pool_window_id_list = []
    req_pool_window_id_list = []
    pool_start_time_list = []
    pool_window_duration_list = []
    
    i = 0
    pool_window_id_list.append(pool_window_id)
    pool_start_time_list.append(pool_start_time)
    pool_window_duration_list.append(pool_duration)
    
    for pickup_datetime in df.tpep_pickup_datetime:
        
        pool_start_time, pool_window_id = getPoolWindow(pickup_datetime, pool_start_time, pool_window_id)
        req_pool_window_id_list.append(pool_window_id)

        #print('pool_window_id_list[',i,'] ',pool_window_id_list[i],' pool_window_id: ',pool_window_id)
        if(pool_window_id_list[i] != pool_window_id):
            pool_window_id_list.append(pool_window_id)
            pool_start_time_list.append(pool_start_time)
            pool_window_duration_list.append(pool_duration)
            i += 1
    
    df['pool_window_id'] = req_pool_window_id_list

    pool_window_data = {'poolingWindowID': pool_window_id_list, 'poolingStartTime': pool_start_time_list, 'poolingDuration': pool_window_duration_list}
    pool_window_df = pd.DataFrame(data=pool_window_data)
    return pool_window_df

### 2.3 Calculate Individual Ride Time

In [ ]:
def getRideTime(destID):
    destObj = df_destinations.loc[(df_destinations['destID']==destID)]
    #print(destObj['timeFromSrc'].values[0])
    return destObj['timeFromSrc'].values[0]

### 2.4 Calculate Individual Ride Distance

In [ ]:
def getRideDistance(destID):
    destObj = df_destinations.loc[(df_destinations['destID']==destID)]
    #print(destObj['distFromSrc'].values[0])
    return destObj['distFromSrc'].values[0]

### 2.5 Calculate Delay

In [ ]:
def getDelay(tripTime):
    #tripTime = (dropoffTime - pickupTime).seconds
    trip_percent = tripTime * max_delay_percent
    return min(trip_percent, max_delay)

### 2.6 Calculate Walking Time

In [ ]:
def getWalkingTime(isWillingToWalk, tripTime):
    if isWillingToWalk==1:
        #tripTime = (dropoffTime - pickupTime).seconds
        trip_percent = tripTime * max_walking_time_percent
    elif isWillingToWalk==0:
        return 0
    return min(trip_percent, max_walking_time)

### 2.7 Calculate Cost for Ride

In [ ]:
def sec_to_min(sec_time):
    return sec_time/60

In [ ]:
def meter_to_mile(meter_distance):
    return meter_distance/1609.34

In [ ]:
def cost_function(isSharing, distance, time):
    #Convert milliseconds to minutes
    time = sec_to_min(time)
    #distance = distance/1000
    #Convert meters to miles
    distance = meter_to_mile(distance)
    if isSharing==True:
        cost_per_mile = 1.00
        cost_per_min = 0.18
        base_fare = 0
        booking_fee = 2.3
        min_fare = 7.3
    else:
        cost_per_mile = 1.80
        cost_per_min = 0.28
        base_fare = 0
        booking_fee = 2.3
        min_fare = 7.3
    #print(distance, time)
    total_fare = (cost_per_mile * distance) + (cost_per_min * time) + base_fare + booking_fee + min_fare
    return total_fare

# Execute Preprocessing

In [ ]:
df = pd.read_csv(filepath)
df_destinations = sql.query('select * from destination',True)

df['tpep_pickup_datetime'] = [datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S') 
                                          for pickup_time in df['tpep_pickup_datetime']]

df['tpep_dropoff_datetime'] = [datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S') 
                                          for dropoff_time in df['tpep_dropoff_datetime']]
import time
start_time = time.time()
df['destID'] = df.apply(lambda x: search_map_dest_id(40.69134374000000000000, 40.88140500000000000000, -74.04164664000000000000, -73.87790573000000000000, x['dropoff_latitude'], x['dropoff_longitude'], 0, 5),axis = 1)
df.loc[df['destID']==False,'destID']=df.loc[df['destID']==False].apply(lambda x: getdestid(x['dropoff_latitude'], x['dropoff_longitude']),axis = 1)
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
df['indvRideTime'] = df.apply(lambda x: getRideTime(x['destID']),axis = 1)
df['indvRideDist'] = df.apply(lambda x: getRideDistance(x['destID']),axis = 1)
df['indvRideCost'] = df.apply(lambda x: cost_function(False,x['trip_distance'],x['indvRideTime']),axis = 1)
df['maxDelay'] = df.apply(lambda x: getDelay(x['indvRideTime']),axis = 1)
df['maxWalkTime'] = df.apply(lambda x: getWalkingTime(x['isWillingToWalk'],x['indvRideTime']),axis = 1)

# 3. DF_Request Table

In [ ]:
def populate_cache_dictionary(df_cache):
    for index, row in df_cache.iterrows():
        key = (row['destID1'], row['destID2'])
        value = (row['time1to2'], row['dist1to2'])
        if key in destinationDict:
            print('Exists', destinationDict[key])
        else:
            #print('Not Exists - Added')
            destinationDict[key] = value

In [ ]:
pool_window_df = poolWindowAssignment()

In [ ]:
df['reqID'] =  df.index + 1
request_df = df[['reqID', 'numberOfPassengers', 'isWillingToWalk', 'destID', 'maxDelay','maxWalkTime','tpep_pickup_datetime','indvRideTime','indvRideDist','indvRideCost','pool_window_id']].copy()

In [ ]:
request_df=request_df.rename(index=str, columns={'tpep_pickup_datetime':'requestTime','trip_distance':'indvRideDist','pool_window_id':'poolingWindowID'})

In [ ]:
request_df

In [ ]:
destinationDict ={}

In [ ]:
df_cache = sql.query('select * from DESTINATION_CACHE',True)
populate_cache_dictionary(df_cache)

In [ ]:
destinationDict

# 4. Insert into SQL DB

In [ ]:
#pool_window_df
sql.insert(pool_window_df,'POOLING_WINDOW');

In [ ]:
sql.insert(request_df,'REQUESTS');

# 4. Shareability Graph

### For each pool

In [ ]:
def getRequestListForEachPoolId():
    max_pool_window_id = df['pool_window_id'].max()
    all_req_list = []
    each_pool_req_list = []
    
    for x in range(1,  max_pool_window_id + 1):
        each_pool_req_list = df.loc[df['pool_window_id'] == x]
        all_req_list.append(each_pool_req_list)
    return all_req_list

In [ ]:
def sharability_graph(pool_request_list):
    
    source_dest = (40.644190, -73.782366)
    taxi_capacity = 4
    graph = []
    possible_match_shared_details_map = {}
    all_req_set = set()
    
    for i in range(pool_request_list.shape[0]):
    #for index, req_A in pool_request_list.iterrows():
        req_A = pool_request_list.iloc[i,:]
        req_id_A = req_A['reqID']
        all_req_set.add(req_id_A)
        dest_id_A = req_A['destID']
        walk_time_A = req_A['maxWalkTime']
        indv_ride_time_A = req_A['indvRideTime']
        max_delay_A = req_A['maxDelay']
        
        for j in range(i+1, pool_request_list.shape[0]):
        #for index+1, req_B in pool_request_list.iterrows():
            
            req_B = pool_request_list.iloc[j,:]
            req_id_B = req_B['reqID']
            all_req_set.add(req_id_B)
            dest_id_B = req_B['destID']
            indv_ride_time_B = req_B['indvRideTime']
            max_delay_B = req_B['maxDelay']
            walk_time_B = req_A['maxWalkTime']
                
            if req_id_A != req_id_B and req_A['passenger_count'] + req_B['passenger_count'] <= taxi_capacity:
                
                poss_dest_A_list = []
                poss_dest_B_list = []
                
                if(apply_euclidean_elimination(req_A, req_B)==False):
                    #print('Euc Fail')
                    continue
                
                poss_dest_A_list , poss_dest_B_list = possibleDestinations(dest_id_A, walk_time_A, dest_id_B, walk_time_B)

                benefit = 0
                   
                object_Dest1, object_Dest2, isReverse = select_route(source_dest, poss_dest_A_list, poss_dest_B_list)

                shared_distance_1to2, shared_time_1to2 = compute_shared_distance_time(object_Dest1, object_Dest2)
                
                if isReverse:
                    #B to A
                    #Check A delay
                    indv_ride_time_Dest2 = indv_ride_time_A
                    max_delay_Dest2 = max_delay_A
                    indv_ride_cost_Dest1 = req_B['indvRideCost']
                    indv_ride_cost_Dest2 = req_A['indvRideCost']
    
                else:
                    #A to B
                    #Check B delay
                    indv_ride_time_Dest2 = indv_ride_time_B
                    max_delay_Dest2 = max_delay_B
                    indv_ride_cost_Dest1 = req_A['indvRideCost']
                    indv_ride_cost_Dest2 = req_B['indvRideCost']
                  
                indv_ride_time_srctoDest1 = object_Dest1['timeFromSrc'].values[0]
                shared_distance_2 = object_Dest1['distFromSrc'].values[0] + shared_distance_1to2
                shared_time_2 = object_Dest1['timeFromSrc'].values[0] + shared_time_1to2
                
                if shared_time_2 <= indv_ride_time_Dest2 + max_delay_Dest2:
                    shared_ride_cost_Dest1 = cost_function(True, object_Dest1['distFromSrc'].values[0], object_Dest1['timeFromSrc'].values[0])
                    shared_ride_cost_Dest2 = cost_function(True, shared_distance_2, shared_time_2)
                    benefit = compute_benefit(indv_ride_cost_Dest1, indv_ride_cost_Dest2, shared_ride_cost_Dest1, shared_ride_cost_Dest2)
                   
                else:
                    #print('Delay Fail')
                    continue
                    
                graph.append((req_id_A, req_id_B, benefit))
                if isReverse:
                    #B to A
                    req_id_1 = req_id_B
                    req_id_2 = req_id_A
                else:
                    #A to B
                    req_id_1 = req_id_A
                    req_id_2 = req_id_B
                possible_match_shared_details_map[str(req_id_1)+'&'+str(req_id_2)] = [object_Dest1['timeFromSrc'].values[0], object_Dest1['distFromSrc'].values[0], shared_ride_cost_Dest1, shared_time_2, shared_distance_2, shared_ride_cost_Dest2, req_A['pool_window_id'], object_Dest1['destID'].values[0], object_Dest2['destID'].values[0]]
            #else:
                #print('Cap Fail')
    sorted_graph = sorted(graph, key=lambda x: x[2],  reverse=True)          
    return sorted_graph, possible_match_shared_details_map, all_req_set

In [ ]:
def possibleDestinations(id1, walk1, id2, walk2):
    list1 = []
    list1.append(id1)
    list2 = []
    list2.append(id2)

    dest_tag = "DEST"
    id1 = int(id1.split(dest_tag)[1])
    id2 = int(id2.split(dest_tag)[1])

    if(walk1 > 0):
        list1 = createDestinationList(dest_tag,id1)

    if(walk2 > 0):
        list2 = createDestinationList(dest_tag,id2)
        
    if(walk1 > 120):
        for i in range(0, len(list1)):
            id0 = int(list1[i].split(dest_tag)[1])
            list1.extend(createDestinationList(dest_tag,id0))

    if(walk2 > 120):
        for i in range(0, len(list2)):
            id0 = int(list2[i].split(dest_tag)[1])
            list2.extend(createDestinationList(dest_tag,id0))

    return list(set(list1)),list(set(list2))

def createDestinationList(dest_tag, dest_id_num):
        list1 = []
        list1.append(dest_tag+str(dest_id_num))
        
        #Corners
        if(dest_id_num == 1):
            list1.append(dest_tag+"31")
            list1.append(dest_tag+"2")
        elif(dest_id_num == 30):
            list1.append(dest_tag+"29")
            list1.append(dest_tag+"60")
        elif(dest_id_num == 3331):
            list1.append(dest_tag+"3332")
            list1.append(dest_tag+"3301")
        elif(dest_id_num == 3360):
            list1.append(dest_tag+"3359")
            list1.append(dest_tag+"3330")

        #North edge
        elif(dest_id_num <= 30):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(id+30))

        #south edge
        elif(dest_id_num >= 3330 and dest_id_num <= 3360):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num-30))

        #west edge
        elif(dest_id_num%30 == 1):
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num-30))
            list1.append(dest_tag+str(dest_id_num+30))

        #east edge
        elif((dest_id_num%30) == 0):
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num+30))
            list1.append(dest_tag+str(dest_id_num-30))
        else:
            list1.append(dest_tag+str(dest_id_num-1))
            list1.append(dest_tag+str(dest_id_num+1))
            list1.append(dest_tag+str(dest_id_num+30))
            list1.append(dest_tag+str(dest_id_num-30))
        return list1

In [ ]:
def apply_euclidean_elimination(reqA, reqB):
    
    #Retrieve destID for request A & B
    destID_A = reqA['destID']
    destID_B = reqB['destID']
    
    #Get coordinates for destID_A and destID_B
    latA = df_destinations.loc[df_destinations['destID']==destID_A].iloc[0]['destLat']
    lonA = df_destinations.loc[df_destinations['destID']==destID_A].iloc[0]['destLong']
    latB = df_destinations.loc[df_destinations['destID']==destID_B].iloc[0]['destLat']
    lonB = df_destinations.loc[df_destinations['destID']==destID_B].iloc[0]['destLong']
    
    #Calculate euclidean distance in meters
    euc_distance = geodesic((latA,lonA), (latB,lonB)).km

    #Using assumed average speed & euclidean distance calculate euclidean time from A to B
    avg_speed = 32
    calc_time_AtoB = (euc_distance/avg_speed)*3600
    
    #Check euclidean conditions for A to B and B to A
    time_StoA = reqA['indvRideTime']
    maxDelayA = reqA['maxDelay']
    time_StoB = reqB['indvRideTime']
    maxDelayB = reqA['maxDelay']
    if (time_StoA + calc_time_AtoB < time_StoB + maxDelayB) and (time_StoB + calc_time_AtoB < time_StoA + maxDelayA):
        return True
    else:
        return False

In [ ]:
#this will return the ID of first destination, ID of second destination, 
#distance from jfk to first destination, time from jfk to first destination
def select_route(jfk, list1, list2):
    #check all pairs
    minDist = sys.maxsize
    final_object_firstDest = df_destinations.loc[df_destinations['destID']==list1[0]]
    final_object_secondDest = df_destinations.loc[df_destinations['destID']==list2[0]]
    isReverse =  False

    #check all pairs
    for i in list1:
        for j in list2:

            aobject = df_destinations.loc[df_destinations['destID']==i]
            bobject = df_destinations.loc[df_destinations['destID']==j]

            a = (aobject["destLat"].values[0], aobject["destLong"].values[0])
            b = (bobject["destLat"].values[0], bobject["destLong"].values[0])

            jfkToA = aobject["distFromSrc"].values[0]
            jfkToB = bobject["distFromSrc"].values[0]
            # jfkToB = geodesic(jfk, b).miles
            AToB = geodesic(a, b).km * 1000

            totalAB = jfkToA+AToB
            totalBA = jfkToB+AToB
            # totalBA = jfkToB+AToB

            #update indices if smaller route found, update minDist 
            if(totalAB < minDist):
                minDist = totalAB
                final_object_firstDest = aobject
                final_object_secondDest = bobject
                isReverse = False

            if(totalBA < minDist):
                minDist = totalBA
                final_object_firstDest = bobject
                final_object_secondDest = aobject
                isReverse = True

    #return indices in order
    return final_object_firstDest, final_object_secondDest, isReverse

In [ ]:
def checkCache(aObject, bObject):
    key = (aObject['destID'].values[0], bObject['destID'].values[0])
    if key in destinationDict:
        #print('FOUND IN CACHE', key)
        value = destinationDict[key]
        return value[0], value[1]
    else:  
        return None, None

In [ ]:
def updateCache(aObject, bObject, time, distance):
    key = (aObject['destID'].values[0], bObject['destID'].values[0])
    value = (time, distance)
    destinationDict[key] = value
    array = [key[0], key[1], value[0],value[1]]
    df_cache= pd.DataFrame(np.array([array]), columns=['destID1', 'destID2', 'time1to2', 'dist1to2'])
    #print('KEY')
    #print(key[0], key[1])
    sql.insert(df_cache,'DESTINATION_CACHE');
    return

In [ ]:
#return distance and time from a to b;
#add this to min_jfk_to_a_dist and min_jfk_to_a_time return from the pervious method
#to compute the total time and distance
def compute_shared_distance_time(aObject, bObject):
    #compute jfk to a, we have that in our dataframe
    time, distance = checkCache(aObject, bObject)
    if time is None and distance is None:
        source = str(aObject["destLat"].values[0])+ ", " + str(aObject["destLong"].values[0])
        destination = str(bObject["destLat"].values[0])+ ", " + str(bObject["destLong"].values[0])
        URL = "https://graphhopper.com/api/1/route?point=" + source + "&point=" + destination + "&vehicle=car&debug=true&key="+graphhoper_key_test+"&type=json"
        r = requests.get(url = URL)
        data = r.json()
        #print(data)
        global api_hits
        api_hits+=1
        #time in seconds
        time = data['paths'][0]['time']/1000
        distance = data['paths'][0]['distance']
        updateCache(aObject, bObject, time, distance)
    return distance, time

In [ ]:
def compute_benefit(indv_ride_cost_Dest1, indv_ride_cost_Dest2, shared_ride_cost_Dest1, shared_ride_cost_Dest2):
    return indv_ride_cost_Dest1+indv_ride_cost_Dest2-shared_ride_cost_Dest1-shared_ride_cost_Dest2

In [ ]:
def optimal_match(sorted_graph):
    g = nx.Graph()
    g.add_weighted_edges_from(sorted_graph)
    optimal_matches = nx.max_weight_matching(g, maxcardinality= False)
    #fair_matches = nx.maximal_matching(g)
    
    return optimal_matches

In [ ]:
def create_data_for_ride_table(optimal_matches, possible_match_shared_details_map, all_req_set, req_df, ride_id_counter):
    
    rides_table_records = []
    for match in optimal_matches:
        match_name = str(match[0])+'&'+str(match[1])
        match_name_rev = str(match[1])+'&'+str(match[0])
        req1_details = []
        req2_details = []
        if match_name in possible_match_shared_details_map:
            match_shared_details = possible_match_shared_details_map[match_name]
            req1_details = [match[0], ride_id_counter, 1, match_shared_details[7], match_shared_details[0], match_shared_details[1] , match_shared_details[2], match_shared_details[6]]
            req2_details = [match[1], ride_id_counter, 1, match_shared_details[8], match_shared_details[3], match_shared_details[4] , match_shared_details[5], match_shared_details[6]]
        elif match_name_rev in possible_match_shared_details_map:
            match_shared_details = possible_match_shared_details_map[match_name_rev]
            req1_details = [match[1], ride_id_counter, 1, match_shared_details[8], match_shared_details[3], match_shared_details[4] , match_shared_details[5], match_shared_details[6]]
            req2_details = [match[0], ride_id_counter, 1, match_shared_details[7], match_shared_details[0], match_shared_details[1] , match_shared_details[2], match_shared_details[6]]
        
        if len(req1_details)> 0 and len(req2_details)>0:
            rides_table_records.append(req1_details)
            rides_table_records.append(req2_details)
            all_req_set.remove(match[0])
            all_req_set.remove(match[1])
            ride_id_counter += 1
            
    for req_id in all_req_set:
        actualDestID = req_df.loc[req_df['reqID'] == req_id, 'destID'].iloc[0]
        indvRideTime = req_df.loc[req_df['reqID'] == req_id, 'indvRideTime'].iloc[0]
        indvRideDist = req_df.loc[req_df['reqID'] == req_id, 'indvRideDist'].iloc[0]
        indvRideCost = req_df.loc[req_df['reqID'] == req_id, 'indvRideCost'].iloc[0]
        poolingWindowID = req_df.loc[req_df['reqID'] == req_id, 'poolingWindowID'].iloc[0]
        req_details = [req_id, ride_id_counter, 0, actualDestID, indvRideTime, indvRideDist, indvRideCost, poolingWindowID]
        rides_table_records.append(req_details)
        ride_id_counter += 1
    
    rides_df = pd.DataFrame(np.array(rides_table_records), columns=['reqID', 'rideID', 'isSharing', 'actualDestID', 'sharedRideTime', 'sharedRideDist', 'sharedRideCost', 'poolingWindowID'])
    
    return rides_df, ride_id_counter

In [ ]:
ride_id_counter = 1
all_req_list = getRequestListForEachPoolId()
graph =[]
api_hits = 0
rides_df = pd.DataFrame(columns=['reqID', 'rideID', 'isSharing', 'actualDestID', 'sharedRideTime', 'sharedRideDist', 'sharedRideCost', 'poolingWindowID'])
print('Number of Pools :', str(len(all_req_list)))
print('BEFORE API HITS', api_hits)
total_time = 0
isTimedOut = False

for pool_index in range(len(all_req_list)):
    pool_request_list = all_req_list[pool_index]
    start_time = time.time()
    sorted_graph, possible_match_shared_details_map, all_req_set = sharability_graph(pool_request_list)
    optimal_matches = optimal_match(sorted_graph)
    rides_df, ride_id_counter = create_data_for_ride_table(optimal_matches, possible_match_shared_details_map, all_req_set, request_df, ride_id_counter)
    elapsed_time = time.time() - start_time
    total_time+=elapsed_time
    if pool_request_list['pool_window_id'].iloc[0]%5 == 0:
        print('Pool ID : ' + str(pool_request_list['pool_window_id'].iloc[0]) + '\tPool Size : ' + str(len(pool_request_list)) + '\t Time : ' + str(elapsed_time))
    
    if isTimedOut==False and total_time >300:
        print('! Timed Out !\t Total Time = ' + str(total_time))
        print('Number of Pools processed = ' + str(pool_request_list['pool_window_id'].iloc[0]))
        isTimedOut = True
    
    rides_df['reqID'] = rides_df['reqID'].astype('int')
    sql.query("""update pooling_window set poolingWindowSize= {}, poolingWindowComputeTime = {} where poolingWindowID = '{}'; """.format(len(pool_request_list), elapsed_time, str(pool_request_list['pool_window_id'].iloc[0])),df_flag=False)
    sql.insert(rides_df,'RIDES');

print('AFTER API HITS', api_hits)
print('Total Time = ' + str(total_time))